In [20]:
import tensorflow as tf
import datetime

In [2]:
(train_image,train_labels),(test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
train_image.shape

(60000, 28, 28)

In [4]:
train_image = tf.expand_dims(train_image,-1)

2021-11-26 13:43:31.881966: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-26 13:43:31.881991: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-26 13:43:31.882006: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mi-HP-ProDesk-680-G6-PCI-Microtower-PC): /proc/driver/nvidia/version does not exist
2021-11-26 13:43:31.882370: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-26 13:43:31.904161: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2899885000 Hz
2021-11-26 13:43:31.904894: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ef78adeb60 executing computations on platfo

In [5]:
test_image = tf.expand_dims(test_image,-1)

In [6]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [7]:
train_image = tf.cast(train_image/255,tf.float32) #计算梯度 都是float类型

In [8]:
test_image = tf.cast(test_image/255,tf.float32)

In [9]:
train_labels = tf.cast(train_labels,tf.int64)

In [10]:
test_labels = tf.cast(test_labels,tf.int64)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image,test_labels))

In [13]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [14]:
dataset = dataset.repeat().shuffle(60000).batch(128)  #混洗 和 批次化 

In [15]:
test_dataset = test_dataset.repeat().batch(128)

In [16]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation="relu",input_shape=(None,None,1)), #任意形状
    tf.keras.layers.Conv2D(32,[3,3],activation="relu"),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10,activation="softmax")
    
    
])

In [18]:
model.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

In [21]:
import os
log_dir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [22]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir,histogram_freq=1,)

In [23]:
model.fit(dataset,
         epochs=5,
         steps_per_epoch=60000//128,
         validation_data = test_dataset,
         validation_steps = 10000//128,
         callbacks = [tensorboard_callback])

Train for 468 steps, validate for 78 steps
Epoch 1/5
  6/468 [..............................] - ETA: 1:18 - loss: 2.3247 - accuracy: 0.0964

2021-11-26 14:11:04.903607: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
2021-11-26 14:11:04.903666: E tensorflow/core/platform/default/device_tracer.cc:70] CUDA error: <unknown>


468/468 [==============================] - 15s 33ms/step - loss: 1.4294 - accuracy: 0.5786 - val_loss: 0.7689 - val_accuracy: 0.7675
Epoch 2/5
468/468 [==============================] - 15s 31ms/step - loss: 0.6322 - accuracy: 0.8047 - val_loss: 0.4899 - val_accuracy: 0.8456
Epoch 3/5
468/468 [==============================] - 15s 31ms/step - loss: 0.4713 - accuracy: 0.8512 - val_loss: 0.4069 - val_accuracy: 0.8749
Epoch 4/5
468/468 [==============================] - 15s 31ms/step - loss: 0.4064 - accuracy: 0.8701 - val_loss: 0.3698 - val_accuracy: 0.8839
Epoch 5/5
468/468 [==============================] - 15s 31ms/step - loss: 0.3668 - accuracy: 0.8831 - val_loss: 0.3304 - val_accuracy: 0.8975


In [24]:
#在notebook 显示界面
%load_ext tensorboard
%matplotlib inline

In [25]:
%tensorboard --logdir logs

In [26]:
#如果想在浏览器打开这个画板  通过终端 在log文件下面 打下tensorboard --logdir logs

In [33]:
#自定义标量
def lr_sche(epoch):
    learning_rate = 0.2
    if epoch>5:
        learning_rate = 0.02
    if epoch>10:
        learning_rate = 0.01
    if epoch>20:
        learning_rate = 0.005
    tf.summary.scalar("learning_rate",data=learning_rate,step=epoch)  #把学习率写入文件夹
    return learning_rate


In [34]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche) #训练过程中控制学习速率

In [35]:
file_writer = tf.summary.create_file_writer(log_dir + "/lr")#写入文件 把学习速率

In [36]:
file_writer.set_as_default()#设为默认的文件编辑器

In [37]:
model.fit(dataset,
         epochs=25,
         steps_per_epoch=60000//128,
         validation_data = test_dataset,
         validation_steps = 10000//128,
         callbacks = [tensorboard_callback,lr_callback])

Train for 468 steps, validate for 78 steps
Epoch 1/25
468/468 [==============================] - 15s 32ms/step - loss: 2.3137 - accuracy: 0.1027 - val_loss: 2.3047 - val_accuracy: 0.1135
Epoch 2/25
468/468 [==============================] - 14s 30ms/step - loss: 2.3137 - accuracy: 0.1034 - val_loss: 2.3064 - val_accuracy: 0.1029
Epoch 3/25
468/468 [==============================] - 15s 32ms/step - loss: 2.3133 - accuracy: 0.1012 - val_loss: 2.3172 - val_accuracy: 0.1010
Epoch 4/25
468/468 [==============================] - 15s 32ms/step - loss: 2.3125 - accuracy: 0.1051 - val_loss: 2.3134 - val_accuracy: 0.1010
Epoch 5/25
468/468 [==============================] - 15s 32ms/step - loss: 2.3120 - accuracy: 0.1050 - val_loss: 2.3243 - val_accuracy: 0.1135
Epoch 6/25
468/468 [==============================] - 14s 30ms/step - loss: 2.3137 - accuracy: 0.1022 - val_loss: 2.3217 - val_accuracy: 0.1010
Epoch 7/25
468/468 [==============================] - 14s 31ms/step - loss: 2.3030 - accuracy

In [ ]:
#自定义循环中使用tensorboard

In [39]:
optimizer = tf.keras.optimizers.Adam()

In [40]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [41]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)  #记录交叉商损失

In [42]:
train_loss = tf.keras.metrics.Mean("train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("train_accuracy")


test_loss = tf.keras.metrics.Mean("test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("test_accuracy")


In [43]:
#定义单步训练  计算损失函数
def train_step(model,images,labels):
    with tf.GradientTape() as t :
        pred = model(images)
        loss_step = loss_func(labels,pred) 
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables)) #优化函数的应用
    train_loss(loss_step)  #自动求值  得到平均
    train_accuracy(labels,pred)

In [44]:
def test_step(model,images,labels):
   
    pred = model(images)
    loss_step = loss_func(labels,pred) 

    test_loss(loss_step)  #自动求值  得到平均
    test_accuracy(labels,pred)

In [45]:
current_time =datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [49]:
train_log_dir = 'logs/gradient_tape' + current_time +'/train'
test_log_dir = 'logs/gradient_tape' + current_time + '/test'
train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [50]:
def train():  #这就是fit内部函数实现机理
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):  #开始迭代每一个数据
            train_step(model,images,labels)
        with train_writer.as_default(): #设置为默认的文件编辑器
            tf.summary.scalar('loss',train_loss.result(),step=epoch) #收集标准值
            tf.summary.scalar('acc',train_accuracy.result(),step=epoch)

        for (batch,(images,labels)) in enumerate(test_dataset):  #每训练一轮  在验证集测试一次
            test_step(model,images,labels)
            print("*",end="")
        with test_writer.as_default(): #设置为默认的文件编辑器
            tf.summary.scalar('loss',test_loss.result(),step=epoch) #收集标准值
            tf.summary.scalar('acc',test_accuracy.result(),step=epoch)
            
            
            
            
        print("Epoch{} test_loss is {},tes_accuracy is {}".format(epoch,
                                                         test_loss.result(),
                                                         test_accuracy.result()
                                                        ))
        
        train_loss.reset_states()#重置变量 避免一次二次bacth相加
        train_accuracy.reset_states()
        test_loss.reset_states()#重置变量 避免一次二次bacth相加
        test_accuracy.reset_states()

In [ ]:
train()